
# JSON Standardization and Flattening
This notebook is responsible for standardizing and flattening JSON files. It reads the raw JSON data from the landing zone, applies a schema for validation, flattens nested structures, and then writes the transformed data as Delta Parquet files.

Key Features:
- Reads JSON files from the landing zone.
- Applies a predefined schema for data validation (ensure the schema file is available at landing/schemachecks/[datasetidentifier]/[datasetidentifier]_schema.json).
- Flattens nested JSON structures for easier querying and analysis.
- Saves the processed data as Delta Parquet files for efficient storage and querying.

## Widget Initialization and Configuration

In [0]:
# Clear all existing widgets to avoid duplication
dbutils.widgets.removeAll()

### triton__flow_plans example (Json)

In [0]:
# Clear all existing widgets to avoid duplication
dbutils.widgets.removeAll()

# Create a dropdown for FileType with predefined options
dbutils.widgets.dropdown("FileType", "json", ["json", "xlsx", "xml"], "File Type")

# Source and Destination Storage Accounts
dbutils.widgets.text("SourceStorageAccount", "dplandingstoragetest", "Source Storage Account")
dbutils.widgets.text("DestinationStorageAccount", "dpuniformstoragetest", "Destination Storage Account")

# Source container and dataset identifier
dbutils.widgets.text("SourceContainer", "landing", "Source Container")
dbutils.widgets.text("SourceDatasetidentifier", "triton__flow_plans", "Source Datasetidentifier")

# Source file and key columns
dbutils.widgets.text("SourceFileName", "triton__flow_plans-202408*", "Source File Name")
dbutils.widgets.text("KeyColumns", "Guid", "Key Columns")

# Feedback column and flattening option
dbutils.widgets.text("FeedbackColumn", "EventTimestamp", "Feedback Column")
dbutils.widgets.text("DepthLevel", "1", "Depth Level")
dbutils.widgets.text("SchemaFolderName", "schemachecks", "Schema Folder Name")

### cpx_so__nomination (Json)

In [0]:
# Clear all existing widgets to avoid duplication
dbutils.widgets.removeAll()

# Create a dropdown for FileType with predefined options
dbutils.widgets.dropdown("FileType", "json", ["json", "xlsx", "xml"], "File Type")

# Source and Destination Storage Accounts
dbutils.widgets.text("SourceStorageAccount", "dplandingstoragetest", "Source Storage Account")
dbutils.widgets.text("DestinationStorageAccount", "dpuniformstoragetest", "Destination Storage Account")

# Source container and dataset identifier
dbutils.widgets.text("SourceContainer", "landing", "Source Container")
dbutils.widgets.text("SourceDatasetidentifier", "cpx_so__nomination", "Source Datasetidentifier")

# Source file and key columns
dbutils.widgets.text("SourceFileName", "cpx_so__nomination-20241127T21*", "Source File Name")
dbutils.widgets.text("KeyColumns", "flows_accountInternal_code, flows_accountExternal_code, flows_location_code, flows_direction,  flows_periods_validityPeriod_begin, flows_periods_validityPeriod_end", "Key Columns")

# Feedback column and flattening option
dbutils.widgets.text("FeedbackColumn", "dateCreated", "Feedback Column")
dbutils.widgets.text("DepthLevel", "6", "Depth Level")
dbutils.widgets.text("SchemaFolderName", "schemachecks", "Schema Folder Name")

### ddp_em__dayahead_flows_nemo (xml)

In [0]:
# Clear all existing widgets to avoid duplication
dbutils.widgets.removeAll()

# Define parameters for the Databricks project with relevant options and defaults
# These widgets allow for user input and dynamic configuration during notebook execution

# Dropdown to select the file type for processing
dbutils.widgets.dropdown("FileType", "xml", ["json", "xlsx", "xml"], "File Type")

# Text widgets for source and destination storage account names
dbutils.widgets.text("SourceStorageAccount", "dplandingstoragetest", "Source Storage Account")
dbutils.widgets.text("DestinationStorageAccount", "dpuniformstoragetest", "Destination Storage Account")

# Source container and dataset identifier
dbutils.widgets.text("SourceContainer", "landing", "Source Container")
dbutils.widgets.text("SourceDatasetidentifier", "ddp_em__dayahead_flows_nemo", "Source Datasetidentifier")

# File name pattern and key columns for data processing
dbutils.widgets.text("SourceFileName", "ddp_em__dayahead_flows_nemo-202405*", "Source File Name")
dbutils.widgets.text("KeyColumns", "TimeSeries_mRID,TimeSeries_Period_timeInterval_start,TimeSeries_Period_Point_position", "Key Columns")

# Additional processing parameters
dbutils.widgets.text("FeedbackColumn", "timeseries_timestamp", "Feedback Column")
dbutils.widgets.text("DepthLevel", "1", "Depth Level")
dbutils.widgets.text("SchemaFolderName", "schemachecks", "Schema Folder Name")
dbutils.widgets.text("XmlRootName", "Schedule_MarketDocument", "Xml Root Name")

### ddp_cm__mfrr_settlement (xml)

In [0]:
# Clear all existing widgets to avoid duplication
dbutils.widgets.removeAll()

# Define parameters for the Databricks project with relevant options and defaults
# These widgets allow for user input and dynamic configuration during notebook execution

# Dropdown to select the file type for processing
dbutils.widgets.dropdown("FileType", "xml", ["json", "xlsx", "xml"], "File Type")

# Text widgets for source and destination storage account names
dbutils.widgets.text("SourceStorageAccount", "dplandingstoragetest", "Source Storage Account")
dbutils.widgets.text("DestinationStorageAccount", "dpuniformstoragetest", "Destination Storage Account")

# Source container and dataset identifier
dbutils.widgets.text("SourceContainer", "landing", "Source Container")
dbutils.widgets.text("SourceDatasetidentifier", "ddp_cm__mfrr_settlement", "Source Datasetidentifier")

# File name pattern and key columns for data processing
dbutils.widgets.text("SourceFileName", "*", "Source File Name")
dbutils.widgets.text("KeyColumns", "mRID,TimeSeries_mRID,TimeSeries_Period_timeInterval_start,TimeSeries_Period_Point_position,TimeSeries_Period_resolution", "Key Columns")

# Additional processing parameters
dbutils.widgets.text("FeedbackColumn", "input_file_name", "Feedback Column")
dbutils.widgets.text("DepthLevel", "", "Depth Level")
dbutils.widgets.text("SchemaFolderName", "schemachecks", "Schema Folder Name")
dbutils.widgets.text("XmlRootName", "ReserveAllocationResult_MarketDocument", "Xml Root Name")

### pluto_pc__units_scadamw (xlsx)

In [0]:
# Clear all existing widgets to avoid duplication
dbutils.widgets.removeAll()

# Create a dropdown for FileType with predefined options
dbutils.widgets.dropdown("FileType", "xlsx", ["json", "xlsx", "xml"], "File Type")

# Source and Destination Storage Accounts
dbutils.widgets.text("SourceStorageAccount", "dplandingstoragetest", "Source Storage Account")
dbutils.widgets.text("DestinationStorageAccount", "dpuniformstoragetest", "Destination Storage Account")

# Source container and dataset identifier
dbutils.widgets.text("SourceContainer", "landing", "Source Container")
dbutils.widgets.text("SourceDatasetidentifier", "pluto_pc__units_scadamw", "Source Datasetidentifier")

# Source file and key columns
dbutils.widgets.text("SourceFileName", "UnitsSCADAMW.xlsx", "Source File Name")
dbutils.widgets.text("KeyColumns", "Unit_GSRN", "Key Columns")

# Feedback column and flattening option
dbutils.widgets.text("SheetName", "Sheet", "Sheet Name")

## Setup

### Package Installation and Management

In [0]:
# ==============================================================
# Setup: Package Installation and Management
# ==============================================================

# Purpose:
# Install and manage required Python packages for the Databricks project.
# Ensure compatibility by specifying exact versions where applicable.

# Uninstall any existing version of a custom utility package (optional)
# Uncomment the line below if you need to remove an older version.
# %pip uninstall databricks-custom-utils -y

# Install necessary packages, including:
# - Custom utilities from a GitHub repository (specific branch or version).
# - Common libraries for parsing SQL, Excel, XML, and syntax highlighting.
%pip install \
    git+https://github.com/Open-Dataplatform/utils-databricks.git@v0.7.1 \
    sqlparse \
    openpyxl \
    lxml \
    xmlschema \
    pygments

"""
Package Details:
- `utils-databricks`: Custom utilities from Open-Dataplatform for Databricks.
- `sqlparse`: A library for parsing and formatting SQL queries.
- `openpyxl`: Used for reading and writing Excel files in XLSX format.
- `lxml`: A library for processing XML and HTML.
- `xmlschema`: Provides validation and conversion tools for XML schemas.
- `pygments`: Syntax highlighter for code snippets.
"""

### Logger Class

In [0]:
from custom_utils.logging.logger import Logger

# Initialize the Logger
logger = Logger(debug=True)

### Initialize notebook and get parameters

In [0]:
from custom_utils.config.config import Config

# Initialize the Config object
config = Config.initialize(dbutils=dbutils, debug=False)

# Unpack parameters into the current scope
config.unpack(globals())

### Verify paths and files

In [0]:
from custom_utils.validation.validation import Validator

# Initialize Validator
validator = Validator(config=config, debug=False)

# Unpack parameters into the current scope
validator.unpack(globals())

# Check for exit flag
validator.check_and_exit()

### Exit the notebook if validate fails

In [0]:
# Exit the notebook explicitly in the final cell
if Validator.exit_notebook:
    print(Validator.exit_notebook_message)
    dbutils.notebook.exit(f"Notebook exited: {Validator.exit_notebook_message}")

## Processing Workflow

### Flattening and Processing

In [0]:
from custom_utils.transformations.dataframe import DataFrameTransformer

# Initialize the DataFrameTransformer class with configuration
transformer = DataFrameTransformer(config=config, debug=False)

# Call the main process_and_flatten_data method
try:
    df_initial, df_flattened = transformer.process_and_flatten_data(depth_level=depth_level)

    # Optionally: Rename the column "Timestamp" to "EventTimestamp" and cast it to timestamp
    if config.source_datasetidentifier == "triton__flow_plans":
        df_flattened = df_flattened \
            .withColumn("Timestamp", col("Timestamp").cast("timestamp")) \
            .withColumnRenamed("Timestamp", "EventTimestamp")

    if config.source_datasetidentifier == "cpx_so__nomination":
        df_flattened = df_flattened \
            .withColumn("dateCreated", col("dateCreated").cast("timestamp")) \
            .withColumn("validityPeriod_begin", col("validityPeriod_begin").cast("timestamp")) \
            .withColumn("validityPeriod_end", col("validityPeriod_end").cast("timestamp")) \
            .withColumn("flows_periods_validityPeriod_begin", col("flows_periods_validityPeriod_begin").cast("timestamp")) \
            .withColumn("flows_periods_validityPeriod_end", col("flows_periods_validityPeriod_end").cast("timestamp"))

    # Display the initial and flattened DataFrames for verification
    print("\nInitial DataFrame:")
    display(df_initial)

    print("Flattened DataFrame:")
    display(df_flattened)

    df_flattened.createOrReplaceTempView("tmp_data")
except Exception as e:
    # Log any errors during processing
    print(f"Error during processing: {str(e)}")

## Quality check 

### Abort if duplicates exists in new data

In [0]:
from custom_utils.quality.quality import DataQualityManager

# Initialize the DataQualityManager class
quality_manager = DataQualityManager(logger=logger, debug=True)

# Describe available quality checks
quality_manager.describe_available_checks()

# Executing Data Quality Checks on Flattened DataFrame
cleaned_data_view = quality_manager.perform_data_quality_checks(
    spark=spark,
    df=df_flattened,
    key_columns=key_columns, # (Union[str, List[str]]): Key columns for partitioning and duplicate checking.
    order_by=feedback_column,  # (Optional[Union[str, List[str]]]): Columns to use for ordering within partitions.
    feedback_column=feedback_column,  # (Optional[str]): Column to use for ordering duplicates; falls back to `key_columns` if None.
    join_column=key_columns, # (Optional[str]): Column to use for referential integrity check
    columns_to_exclude=["input_file_name"], # (Optional[List[str]]): Columns to exclude from the final DataFrame.
    use_python=False  # (Optional[bool]): If True, uses Python syntax for operations; defaults to SQL.
)

## Write

### Unified Data Storage Management: Table Creation and Data Merging

In [0]:
from custom_utils.catalog.catalog_utils import DataStorageManager

# Initialize the DataStorageManager class
storage_manager = DataStorageManager(logger=logger, debug=True)

storage_manager.manage_data_operation(
    spark=spark,
    dbutils=dbutils,
    cleaned_data_view=cleaned_data_view,  # The view you want to merge
    key_columns=key_columns,  # Key columns to use for merging
    destination_folder_path=destination_data_folder_path,  # Destination folder path
    destination_environment=destination_environment,  # The database name
    source_datasetidentifier=source_datasetidentifier,  # The table name
    use_python=False  # Set to True to use Python DataFrame operations; False for SQL
)

## Finishing

### Return period (from_datetime, to_datetime) covered by data read

In [0]:
# Run the main function to generate feedback timestamps
notebook_output = storage_manager.generate_feedback_timestamps(
    spark=spark, 
    view_name=cleaned_data_view, 
    feedback_column=feedback_column,
    key_columns=key_columns
)

## Exit the notebook

In [0]:
# Use dbutils to exit the notebook
dbutils.notebook.exit(notebook_output)